# Faster R-CNN annotation

In [ ]:
import os
import sys
import itertools
import random
import datetime

import numpy as np
import cv2
import pandas as pd
import json

# make a folder for everything we will need to download (except the images that will be annotated)
!mkdir download


## Annotating frames from a movie
If you wish to annotate frames from a movie, first convert them into images and save them in a new folder called 'images' in the same directory as this. **If your data is already saved as images, skip this step and don't run the following cell.**
You should specify the filepath of your movie you want to annotate as the 

```
video_path 
```

variable below

In [ ]:
! mkdir images
video_path = "worm_pump02.mp4"
vid = cv2.VideoCapture(video_path)
i = 1
while vid.isOpened():
    ret, image_np = vid.read()
    if ret:
        frame_name = 'img' + str(i) + '.jpg'
        cv2.imwrite(os.path.join("images", frame_name), image_np)
    else:
        break
    print("Processing frame no %s" % i)
    i += 1
vid.release()

## Annotating images
Once your data is in image form, we will use Innotater to annotate it. If you did not run the code above to convert a movie to a set of images, make a folder called "images" in the same folder as this and upload your images there. **To upload your images**, first go to the Jupyter home page or click on the Jupyter icon in the upper left of the notebook. You'll see the directory of the binder, and you can make a new folder and upload images to it using the 'New' dropdown menu and the 'Upload' button in the right corner of the home page. **Make sure the images are named as imgxx.jpg, where xx is a number**. For other image extensions, e.g. .png, you can replace JPG with png in the code below, but note that we have previously had problems with tensorflow not recognizing .png files during training.

Depending on your needs, you may not need to annotate all that many images. Here we only demonstrate fine-tuning a pre-existing model, and while we used between ~100 - 1000 annotations for our models, if you want very high accuracy, you may need to annotate more images.

Once you run the code cell below, you will see an interface for annotation appear. Draw boxes around the types of objects you wish to annotate ('classes'). By default, the classes we use below are those in our paper, 'worm' and 'egg'. Advance to the next image by clicking ‘Next’ or pressing ‘n’ on the keyboard (provided the annotation tool has focus).

In [ ]:
from jupyter_innotater import *
import glob

img_path = './images'
worm_image_files = glob.glob('./images/*.JPG')
worm_image_files.sort(key=os.path.getmtime)
worm_image_files = [os.path.split(file)[1] for file in worm_image_files]
worm_image_files = worm_image_files.reverse()

# 'Repeats' is the maximum number of objects you expect to annotate in each image
repeats = 7
# Feel free to modify the classes, or types of objects, you want to identify
classes = ['worm', 'egg']

# Binary flag to indicate an image should be excluded from dataset
targets_exclude = np.zeros((len(worm_image_files), 1), dtype='int') 

# set up arrays to load annotation info into
targets_classes = np.zeros((len(worm_image_files), len(classes)*repeats), dtype='int')
targets_bboxes = np.zeros((len(worm_image_files), len(classes)*repeats, 4), dtype='int') # (xmin,ymin,w,h) for each animal

Innotater(
    [
        ImageInnotation(worm_image_files, path='./images'), # Display the image itself
        TextInnotation(worm_image_files, multiline=False) # Display the image filename
    ],
    [
        BinaryClassInnotation(targets_exclude, name='Exclude'), # Checkbox
        RepeatInnotation(
            (BoundingBoxInnotation, targets_bboxes), # Individual animal bounding box
            (MultiClassInnotation, targets_classes,
                {'name':'object', 'classes':classes, 'dropdown':True}), # Per-annotation dropdown
            max_repeats=len(classes)*repeats, min_repeats=1
        )
    ]
)

## Save annotations
Here we'll prepare the annotations, save them as a csv file, and build a label map from our list of classes. We will split up annotations now so that the precision and accuracy of the model can be tested later. We choose images at random and choose a 90/ 10 train/test split for the data. Note that tensorflow uses the tfrecord format as input to our model. We won't convert to tfrecord format here, as it requires installing Tensorflow, but you can follow our Google CoLab notebook to continue the training process for a Faster R-CNN.

In [ ]:
# reshape data so that the structure is flat
flat_bboxes = np.reshape(targets_bboxes, (-1, 4))
flat_classes = np.reshape(targets_classes, (-1,))
flat_classes = [classes[x] for x in flat_classes]

# Grab widths and heights for each image 
widths = np.zeros((len(worm_image_files),), dtype='int')
heights = np.zeros((len(worm_image_files),), dtype='int')
for i, im_file in enumerate(worm_image_files):
    im = cv2.imread(os.path.join('./images', im_file))
    heights[i], widths[i], d = im.shape
            
# it can be helpful to have annotations stored in a generic csv as well, so we'll prepare annotations and do that here.
data = {'filename': np.repeat(worm_image_files, len(classes)*repeats), 'xmin': flat_bboxes[:,0], 'ymin': flat_bboxes[:,1], 
        'xmax': np.add(flat_bboxes[:,0], flat_bboxes[:,2]), 'ymax': np.add(flat_bboxes[:,1], flat_bboxes[:,3]), 
        'width': np.repeat(widths, len(classes)*repeats), 'height':np.repeat(heights, len(classes)*repeats),
        'exclude': np.repeat(np.reshape(targets_exclude, (-1,)), len(classes)*repeats), 'class': flat_classes}
df = pd.DataFrame(data)
# screen out any un-annotated frames or any images marked by the annotator as 'Exclude'
df_annotated = df[(df['exclude'] == 0) & (df['xmin'] != df['xmax'])]

# if you want to change the train/ test split, you can decrease number of training images (which will increase the testing images) by decreasing this. It must be between 0 and 1
train_split = 0.9 
annotated_images = df_annotated['filename'].unique()
num_train_images = int(train_split*len(annotated_images))
train_images = random.sample(list(annotated_images), num_train_images)
test_images = list(set(annotated_images).difference(set(train_images)))
test_idx = df_annotated['filename'].isin(test_images)
test_or_train = ['test' if test_i else 'train' for test_i in test_idx]
df_annotated.insert(9, 'test_or_train', test_or_train)

# save csv to annotations directory
csv_filepath = './download/bounding_boxes.csv'
df_annotated.to_csv(csv_filepath, index=False)

# next, make a label file so we know how to map the names of classes to a number. 'worm' will map to 1, 'egg' will map to 2, and so on
def write_label_map(classes, label_map_path):
    s = ' '
    for ID, name in enumerate(classes):
        out = ''
        out += 'item' + s + '{\n'
        out += '\t' + 'id:' + s + (str(ID+1)) + '\n'
        out += '\t' + 'name:' + s + '\'' + name + '\'' + '\n'
        out += '}\n\n'
        with open(label_map_path, 'a') as f:
            f.write(out)
        
label_map_path = './download/label_map.pbtxt'
write_label_map(classes, label_map_path)

## Download your files
Once your label_map.pbtxt and bounding_boxes.csv files have been successfully generated, **download the 'download' folder to your local computer**. If you converted a video to images in order to annotate, you also need to download the 'images' folder!